In [56]:
import numpy as np
from itertools import permutations

In [55]:
## STEP 1: Profit allocation, reference: paper by Nardelli et al.


# Define electricity consumption (El) and production (Ep) for each customer (i)
El = np.array([300, 200, 400, 250])  # Total electricity consumption for customers
Ep = np.array([200, 300, 350, 150])  # Total electricity production for customers

Cg = 0.12  # Cost of purchasing electricity from the grid
Cr = 0.15  # Price at which electricity is sold to an aggregator
delta = 0.5  # Proportion of excess electricity sold to aggregator (0 <= delta <= 1)

# Calculate marginal contributions for each customer i
def calculate_marginal_contributions(El, Ep, Cg, Cr, delta):
    n = len(El)
    MCs = np.zeros(n)
    
    for i in range(n):
        if Ep[i] >= El[i]:  # When prosumer (Ep ≥ El)
            # MC calculation when excess electricity can be sold to aggregator
            if i in range(n):  # Assuming i is a prosumer (buys electricity)
                if Ep[i] >= El[i]:
                    MCs[i] = El[i] * Cg
                else:
                    MCs[i] = (Ep[i] * Cg) - (El[i] * Cg)
                    
                if Ep[i] >= El[i]:
                    MCs[i] += delta * (Ep[i] - El[i]) * Cr
            
        else:  # When consumer (Ep < El)
            # MC calculation when community needs to buy more electricity
            if i in range(n):  # Assuming i is a consumer (sells excess electricity)
                MCs[i] = (Ep[i] - El[i]) * Cg
            
    # Normalize MCs to avoid negative values
    MCs -= np.min(MCs)
    
    return MCs

# Calculate profit allocations (φi,new) for each customer i
def calculate_profit_allocations(El, Ep, Cg, Cr, delta):
    MCs = calculate_marginal_contributions(El, Ep, Cg, Cr, delta)
    n = len(El)
    
    # Sum of MCs for normalization
    sum_MC = np.sum(MCs)
    
    allocations = np.zeros(n)
    
    for i in range(n):
        if Ep[i] >= El[i]:  # When prosumer (Ep ≥ El)
            allocations[i] = MCs[i] / sum_MC * (El[i] * Cg + delta * (Ep[i] - El[i]) * Cr)
        else:  # When consumer (Ep < El)
            allocations[i] = MCs[i] / sum_MC * Ep[i] * Cg
            
    return allocations

# Example usage:
profit_allocations = calculate_profit_allocations(El, Ep, Cg, Cr, delta)
print("Profit Allocations:", profit_allocations)


Profit Allocations: [ 0.         27.68181818  5.09090909  0.        ]


In [76]:
# Probably useless

# Define electricity consumption (El) and production (Ep) for each customer (i)
El = np.array([300, 200, 400, 250])  # Total electricity consumption for customers
Ep = np.array([200, 3000, 350, 150])  # Total electricity production for customers

# Calculate Shapley value-based fair energy allocation
def calculate_shapley_allocation(El, Ep):
    n = len(El)
    coalitions = list(range(n))
    shapley_values = np.zeros(n)
    
    for i in range(n):
        # Calculate marginal contributions for player i
        marginal_contributions = []
        for coalition in permutations(coalitions):
            coalition = list(coalition)
            if i not in coalition:
                continue
            v_without_i = sum(Ep[j] for j in coalition if j != i) * Cg
            v_with_i = sum(Ep[j] for j in coalition) * Cg
            marginal_contribution = v_with_i - v_without_i
            marginal_contributions.append(marginal_contribution)
        
        # Calculate Shapley value for player i
        shapley_values[i] = np.mean(marginal_contributions)
    
    # Normalize Shapley values to sum up to total profit
    total_profit = sum(Ep) * Cg
    shapley_values /= np.sum(shapley_values)
    shapley_allocations = shapley_values * total_profit
    
    return shapley_allocations

# Example use
Cg = 0.12  # Cost of purchasing electricity from the grid
shapley_allocations = calculate_shapley_allocation(El, Ep)

# Print the Shapley value-based energy allocations
print("Shapley Value-based Energy Allocations:")
for i in range(len(El)):
    print(f"Customer {i}: {shapley_allocations[i]} units of electricity")


Shapley Value-based Energy Allocations:
Customer 0: 24.0 units of electricity
Customer 1: 360.0 units of electricity
Customer 2: 42.0 units of electricity
Customer 3: 18.0 units of electricity


In [54]:
## STEP 2: Simple optimization of energy allocation using the same approach (Shapley value) as for
# profit allocation in step 1

import itertools

class MicrogridNode:
    def __init__(self, id, supply, demand):
        self.id = id
        self.supply = supply  # Energy supply of the node
        self.demand = demand  # Energy demand of the node
        self.utility = 0      # Utility of the node after allocation
    
    def __repr__(self):
        return f"Node {self.id} (Supply: {self.supply}, Demand: {self.demand}, Utility: {self.utility})"

def calculate_shapley_value_demand(nodes, total_energy):
    num_nodes = len(nodes)
    shapley_values = [0] * num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_demand = sum(node.demand for node in coalition)
            
            if coalition_demand <= total_energy:
                # Calculate marginal contribution of node[i] to the coalition
                if i == 0:
                    marginal_contribution = coalition_demand
                else:
                    prev_coalition_demand = sum(node.demand for node in perm[:i])
                    marginal_contribution = coalition_demand - prev_coalition_demand
                
                # Calculate Shapley value for node[i] based on demand
                shapley_values[perm[i].id - 1] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    
    return normalized_shapley_values

def allocate_energy_packets_demand(nodes, total_energy):
    shapley_values = calculate_shapley_value_demand(nodes, total_energy)
    
    # Allocate energy packets based on normalized Shapley values
    for i, node in enumerate(nodes):
        node.utility = shapley_values[i]
    
    return nodes

# Example usage
if __name__ == "__main__":
    # Define microgrid nodes (id, supply, demand)
    nodes = [
        MicrogridNode(1, 20, 6),
        MicrogridNode(2, 15, 12),
        MicrogridNode(3, 25, 15),
        MicrogridNode(4, 18, 15)
    ]
    
    total_available_energy = 78  # Total energy packets available for allocation
    
    # Allocate energy packets using the Shapley value approach based on demand
    allocated_nodes = allocate_energy_packets_demand(nodes, total_available_energy)
    
    # Display allocated nodes with updated utility values
    print("Allocated Nodes (Based on Demand):")
    for node in allocated_nodes:
        print(node)

    # Calculate and display the total allocated energy (sum of utilities)
    total_allocated_energy = sum(node.utility for node in allocated_nodes)
    print(f"Total Allocated Energy: {total_allocated_energy}")


Allocated Nodes (Based on Demand):
Node 1 (Supply: 20, Demand: 6, Utility: 9.75)
Node 2 (Supply: 15, Demand: 12, Utility: 19.5)
Node 3 (Supply: 25, Demand: 15, Utility: 24.375)
Node 4 (Supply: 18, Demand: 15, Utility: 24.375)
Total Allocated Energy: 78.0


In [52]:
# STEP 3: Refined code (energy allocation) for the final work.
# Each household uses their own energy first. Surplus energy is allocated between
# all of the households that have an energy deficit, using Shapley values for fair distribution
# TODO: fix the issue with exactly the same marginal contributions of each node!

import itertools

class MicrogridNode:
    def __init__(self, id, supply, demand):
        self.id = id
        self.supply = supply  # Energy supply of the node
        self.demand = demand  # Energy demand of the node
        self.utility = 0      # Utility of the node after allocation
        self.balance = 0      # Energy balance (surplus or deficit)
    
    def __repr__(self):
        return f"Node {self.id} (Supply: {self.supply}, Demand: {self.demand}, Utility: {self.utility}, Balance: {self.balance})"


def calculate_shapley_value_buy(nodes, total_energy):
    num_nodes = len(nodes)
    shapley_values = [0] * num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    if not permutations:
        print("No premutations")
        return shapley_values  # No valid permutations, return zero Shapley values
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_demand = sum(node.supply - node.demand for node in coalition)
            
            #if coalition_demand <= total_energy:
            # Calculate marginal contribution of node[i] to the coalition
            if i == 0:
                marginal_contribution = coalition_demand
            else:
                prev_coalition_demand = sum(node.supply - node.demand for node in perm[:i])
                marginal_contribution = coalition_demand - prev_coalition_demand
        
            # Calculate Shapley value for node[i] based on demand
            shapley_values[i] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    if shapley_sum != 0:
        normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    else:
        print("Shapley sum == 0")
        normalized_shapley_values = [0] * num_nodes
    
    return normalized_shapley_values

def calculate_shapley_value_sell(nodes, total_energy):
    num_nodes = len(nodes)
    shapley_values = [0] * num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    if not permutations:
        print("No permutations")
        return shapley_values  # No valid permutations, return zero Shapley values
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_balance = sum(node.balance for node in coalition)
            
            #if abs(coalition_balance) <= total_energy:
            # Calculate marginal contribution of node[i] to the coalition
            if i == 0:
                marginal_contribution = abs(coalition_balance)
            else:
                prev_coalition_balance = sum(node.balance for node in perm[:i])
                marginal_contribution = abs(coalition_balance - prev_coalition_balance)
            
            # Calculate Shapley value for node[i] based on balance
            shapley_values[i] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    if shapley_sum != 0:
        normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    else:
        normalized_shapley_values = [0] * num_nodes
    
    return normalized_shapley_values


def allocate_energy_packets_demand(nodes):

    for node in nodes:
        node.balance = node.supply - node.demand

    
    # Determine buying order based on Shapley values of surplus nodes
    surplus_nodes = [node for node in nodes if node.balance > 0]
    sum_surplus = 0
    for surplus_node in surplus_nodes:
        sum_surplus += surplus_node.balance

    deficit_nodes = [node for node in nodes if node.balance < 0]
    sum_deficit = 0
    for deficit_node in deficit_nodes:
        sum_deficit -= deficit_node.balance

    trade_volume = min(sum_surplus, sum_deficit)
    print("Trade volume:", trade_volume)

    if trade_volume <= 0:
        return nodes
    
    shapley_values_buy = calculate_shapley_value_buy(deficit_nodes, trade_volume)
    shapley_values_sell = calculate_shapley_value_sell(surplus_nodes, trade_volume)

    
    # Update utility based on final energy balance after trading
    for i, node in enumerate(deficit_nodes):
        node.utility = node.supply + shapley_values_buy[i]
        node.balance = node.utility - node.demand
        #if shapley_values_buy[i] > 0:
            #trade_amount = shapley_values_buy[i] * trade_volume
            #for j, selling_node in enumerate(surplus_nodes):
            #    if shapley_values_sell[j] > 0:
            #        trade_amount -= min(shapley_values_sell[j] * trade_volume, selling_node.balance)
            #        selling_node.balance -= min(shapley_values_sell[j] * trade_volume, selling_node.balance)
            #        if trade_amount <= 0:
            #            break
            #ode.balance += trade_amount
            #print("Trade amount:", trade_amount)
    for i, node in enumerate(surplus_nodes):
        node.utility = node.supply - shapley_values_sell[i]
        node.balance = node.utility - node.demand

    # Update utility values based on final energy balance
    #for node in nodes:
    #    node.utility = node.supply - node.balance
    
    return nodes, shapley_values_buy, shapley_values_sell

# Example usage
if __name__ == "__main__":
    # Define microgrid nodes (id, supply, demand)
    nodes = [
        MicrogridNode(1, 18, 5),
        MicrogridNode(2, 20, 6),
        MicrogridNode(3, 15, 17),
        MicrogridNode(4, 26, 21),
        MicrogridNode(5, 24, 29),
        MicrogridNode(6, 25, 29),
    ]
    
    #total_available_energy = 78  # Total energy packets available for allocation
    
    # Allocate energy packets and simulate energy trading based on surplus Shapley values
    allocated_nodes, shapley_values_buy, shapley_values_sell = allocate_energy_packets_demand(nodes)
    
    # Display allocated nodes with updated utility values and balance
    print("Allocated Nodes (Based on Surplus Shapley-based Energy Trading):")
    for node in allocated_nodes:
        print(node)

    print("Shapley values:")
    print(shapley_values_buy)
    print(shapley_values_sell)

    # Calculate and display the total allocated energy (sum of utilities)
    total_allocated_energy = sum(node.utility for node in allocated_nodes)
    print(f"Total Allocated Energy: {total_allocated_energy}")


Trade volume: 11
Allocated Nodes (Based on Surplus Shapley-based Energy Trading):
Node 1 (Supply: 18, Demand: 5, Utility: 14.333333333333332, Balance: 9.333333333333332)
Node 2 (Supply: 20, Demand: 6, Utility: 16.333333333333332, Balance: 10.333333333333332)
Node 3 (Supply: 15, Demand: 17, Utility: 18.666666666666664, Balance: 1.6666666666666643)
Node 4 (Supply: 26, Demand: 21, Utility: 22.333333333333332, Balance: 1.3333333333333321)
Node 5 (Supply: 24, Demand: 29, Utility: 27.666666666666668, Balance: -1.3333333333333321)
Node 6 (Supply: 25, Demand: 29, Utility: 28.666666666666668, Balance: -0.33333333333333215)
Shapley values:
[3.666666666666666, 3.666666666666667, 3.666666666666667]
[3.666666666666667, 3.666666666666667, 3.6666666666666665]
Total Allocated Energy: 128.0
